<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"></ul></div>

In [1]:
%matplotlib inline

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [4]:
cookies_validate = pd.read_csv("../data/cookies_validate.csv")
cookies = pd.read_csv("../data/cookies.csv")

In [5]:
cookies

,sugar to flour ratio,sugar index,bake temp,chill time,calories,density,pH,grams baking soda,bake time,quality,butter type,weight,diameter,mixins,crunch factor,aesthetic appeal
0,0.25,9.5,300,15.0,136.0,0.99367,8.10,0.44,12.1,8,melted,15.2,7,raisins,1.30,3
1,0.23,3.3,520,34.0,113.0,0.99429,8.16,0.48,8.4,7,melted,12.4,7,raisins,1.71,3
2,0.18,1.9,360,33.0,106.0,0.98746,8.21,0.83,14.0,9,melted,9.4,7,"nuts, chocolate",1.78,3
3,0.18,10.5,490,41.0,124.0,0.99630,8.14,0.35,10.5,7,melted,12.2,7,chocolate,1.59,3
4,0.24,2.4,770,6.0,33.0,0.99740,8.09,0.57,9.4,5,cubed,19.8,7,"nuts, oats, chocolate",1.30,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5193,0.66,2.0,390,15.0,88.0,0.99357,8.66,0.60,11.5,6,cubed,11.6,7,peanut butter,1.36,3
5194,0.16,1.5,490,10.0,97.0,0.99170,8.48,0.44,10.7,8,melted,10.0,7,raisins,1.80,3
5195,0.25,14.3,480,79.0,149.0,0.99750,8.12,0.37,9.7,7,melted,13.4,7,"chocolate, oats",1.07,3
5196,0.33,1.4,560,35.0,136.0,0.99380,8.63,0.78,10.3,8,melted,13.8,7,chocolate,1.77,3


In [6]:
onehot = (pd.get_dummies(cookies['mixins'].str.split(
    pat=",", expand=True))).groupby(lambda x: x.split('_')[-1], axis=1).sum()
onehot
onehot.columns = ['chocolate_1', 'oats_1', 'peanut butter_1', 'raisins_1', 'chocolate_2',
       'nuts_1', 'oats_2', 'peanut butter_2', 'raisins_2']
onehot["chocolate"] = onehot["chocolate_1"] + onehot["chocolate_2"]
onehot["oats"] = onehot["oats_1"] + onehot["oats_2"]
onehot["peanut_butter"] = onehot["peanut butter_1"] + onehot["peanut butter_2"]
onehot["raisins"] = onehot["raisins_1"] + onehot["raisins_2"]
onehot["nuts"] = onehot["nuts_1"]
onehot.drop(columns = ['chocolate_1', 'oats_1', 'peanut butter_1', 'raisins_1', 'chocolate_2',
       'nuts_1', 'oats_2', 'peanut butter_2', 'raisins_2'],inplace=True)
onehot

,chocolate,oats,peanut_butter,raisins,nuts
0,0,0,0,1,0
1,0,0,0,1,0
2,1,0,0,0,1
3,1,0,0,0,0
4,1,1,0,0,1
...,...,...,...,...,...
5193,0,0,1,0,0
5194,0,0,0,1,0
5195,1,1,0,0,0
5196,1,0,0,0,0


In [33]:
cookies = pd.concat([cookies, onehot], axis=1)

In [34]:
cookies = cookies.drop(columns=['sugar to flour ratio','density','pH','diameter','crunch factor','aesthetic appeal', 'mixins'],axis=1).reset_index(drop = True)

In [42]:
cookies = cookies.dropna()

In [43]:
cookies = pd.get_dummies(cookies, drop_first=True)

In [46]:
cookies['butter type_melted'] = cookies['butter type_melted'].astype(int)

In [47]:
cookies.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5183 entries, 0 to 5197
Data columns (total 14 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   sugar index         5183 non-null   float64
 1   bake temp           5183 non-null   int64  
 2   chill time          5183 non-null   float64
 3   calories            5183 non-null   float64
 4   grams baking soda   5183 non-null   float64
 5   bake time           5183 non-null   float64
 6   quality             5183 non-null   int64  
 7   weight              5183 non-null   float64
 8   chocolate           5183 non-null   uint8  
 9   oats                5183 non-null   uint8  
 10  peanut_butter       5183 non-null   uint8  
 11  raisins             5183 non-null   uint8  
 12  nuts                5183 non-null   uint8  
 13  butter type_melted  5183 non-null   int64  
dtypes: float64(6), int64(3), uint8(5)
memory usage: 430.2 KB


--------------------------------------------------------------------------------

In [9]:
cookies_fixed = pd.read_csv("../data/cookies_fixed.csv",index_col=0)

In [10]:
cookies_fixed.head()

,sugar index,bake temp,chill time,calories,grams baking soda,bake time,quality,weight,chocolate,oats,peanut_butter,raisins,nuts,butter type_melted
0,9.5,300,15.0,136.0,0.44,12.1,8,15.2,0,0,0,1,0,1
1,3.3,520,34.0,113.0,0.48,8.4,7,12.4,0,0,0,1,0,1
2,1.9,360,33.0,106.0,0.83,14.0,9,9.4,1,0,0,0,1,1
3,10.5,490,41.0,124.0,0.35,10.5,7,12.2,1,0,0,0,0,1
4,2.4,770,6.0,33.0,0.57,9.4,5,19.8,1,1,0,0,1,0


-----------------------------------------------------------------------------------------------------------------

In [11]:
cookies_validate = pd.read_csv("../data/cookies_validate.csv")

In [13]:
cookies_validate.head()

,id,sugar to flour ratio,sugar index,bake temp,chill time,calories,density,pH,grams baking soda,bake time,quality,butter type,weight,diameter,mixins,crunch factor,aesthetic appeal
0,14,0.62,19.25,400.0,41.0,172.0,1.0002,7.98,0.67,9.7,0,melted,16.6,7,"nuts, chocolate",1.85,3
1,26,0.35,1.00,520.0,35.0,146.0,0.9930,8.45,0.44,10.0,0,melted,13.8,7,chocolate,1.43,3
2,30,0.39,10.40,440.0,20.0,142.0,0.9974,8.20,0.53,10.0,0,melted,17.0,7,chocolate,1.57,3
3,52,0.33,1.10,570.0,21.0,82.0,0.9910,8.32,0.46,10.9,0,melted,12.4,7,"chocolate, oats",1.44,3
4,100,0.37,13.50,600.0,52.0,192.0,0.9975,8.00,0.44,9.1,0,melted,14.8,7,chocolate,1.51,3


In [15]:
onehot_valid = (pd.get_dummies(cookies_validate['mixins'].str.split(
    pat=",", expand=True))).groupby(lambda x: x.split('_')[-1], axis=1).sum()
onehot_valid

,chocolate,oats,peanut butter,chocolate,nuts,peanut butter,raisins
0,1,0,0,0,1,0,0
1,0,0,0,1,0,0,0
2,0,0,0,1,0,0,0
3,0,1,0,1,0,0,0
4,0,0,0,1,0,0,0
...,...,...,...,...,...,...,...
774,1,1,0,0,1,0,0
775,0,1,0,0,1,0,0
776,0,1,0,0,1,0,0
777,0,0,0,0,0,1,0


In [16]:
onehot_valid.columns = ['chocolate_1', 'oats_1', 'peanut butter_1', 'chocolate_2',
       'nuts_1', 'peanut butter_2', 'raisins_1']


onehot_valid["chocolate"] = onehot_valid["chocolate_1"] + onehot_valid["chocolate_2"]

onehot_valid["oats"] = onehot_valid["oats_1"]

onehot_valid["peanut_butter"] = onehot_valid["peanut butter_1"] + onehot_valid["peanut butter_2"]

onehot_valid["raisins"] = onehot_valid["raisins_1"]

onehot_valid["nuts"] = onehot_valid["nuts_1"]


onehot_valid.drop(columns = ['chocolate_1', 'oats_1', 'peanut butter_1', 'chocolate_2',
       'nuts_1', 'peanut butter_2', 'raisins_1'],inplace=True)

onehot_valid

,chocolate,oats,peanut_butter,raisins,nuts
0,1,0,0,0,1
1,1,0,0,0,0
2,1,0,0,0,0
3,1,1,0,0,0
4,1,0,0,0,0
...,...,...,...,...,...
774,1,1,0,0,1
775,0,1,0,0,1
776,0,1,0,0,1
777,0,0,1,0,0


In [18]:
onehot_valid.describe()

,chocolate,oats,peanut_butter,raisins,nuts
count,779.000000,779.000000,779.000000,779.000000,779.000000
mean,0.695764,0.189987,0.006418,0.293967,0.154044
std,0.460379,0.392543,0.079909,0.455870,0.361223
min,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.000000,0.000000,0.000000,0.000000,0.000000
50%,1.000000,0.000000,0.000000,0.000000,0.000000
75%,1.000000,0.000000,0.000000,1.000000,0.000000
max,1.000000,1.000000,1.000000,1.000000,1.000000


In [19]:
cookies_validate = pd.concat([cookies_validate, onehot_valid], axis=1)

In [20]:
cookies_validate.isna().sum()

id                      0
sugar to flour ratio    0
sugar index             0
bake temp               0
chill time              0
calories                0
density                 0
pH                      0
grams baking soda       0
bake time               0
quality                 0
butter type             0
weight                  0
diameter                0
mixins                  0
crunch factor           0
aesthetic appeal        0
chocolate               0
oats                    0
peanut_butter           0
raisins                 0
nuts                    0
dtype: int64

In [22]:
cookies_validate = cookies_validate.drop(columns=['id','sugar to flour ratio','density','pH','diameter','crunch factor','aesthetic appeal', 'mixins'],axis=1).reset_index(drop = True)

In [23]:
cookies_validate = pd.get_dummies(cookies_validate, drop_first=True)

In [25]:
cookies_validate.dtypes

sugar index           float64
bake temp             float64
chill time            float64
calories              float64
grams baking soda     float64
bake time             float64
quality                 int64
weight                float64
chocolate               uint8
oats                    uint8
peanut_butter           uint8
raisins                 uint8
nuts                    uint8
butter type_melted      uint8
dtype: object

In [26]:
cookies_fixed.dtypes

sugar index           float64
bake temp               int64
chill time            float64
calories              float64
grams baking soda     float64
bake time             float64
quality                 int64
weight                float64
chocolate               int64
oats                    int64
peanut_butter           int64
raisins                 int64
nuts                    int64
butter type_melted      int64
dtype: object

In [27]:
cookies_validate.to_csv("../data/cookies_validate_fixed.csv")